<a href="https://colab.research.google.com/github/brianpyai/chatbot/blob/main/gemma_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio transformers
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.4 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=55

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:

# Linux and Mac


from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "LoneStriker/OrcaGemma-2B-8.0bpw-h8-exl2" #@param ["LoneStriker/OrcaGemma-2B-8.0bpw-h8-exl2","LoneStriker/Gemmalpaca-2B-8.0bpw-h8-exl2","LoneStriker/zephyr-7b-gemma-v0.1-GGUF","LoneStriker/openbuddy-gemma-7b-v19.1-4k-8.0bpw-h8-exl2","LoneStriker/gemma-2b","unsloth/gemma-7b" ,"HuggingFaceH4/zephyr-7b-gemma-v0.1","openchat/openchat-3.5-0106-gemma","google/gemma-7b","google/gemma-2b","ccrains/larson-gemma-2b-chinese-v0.1" ,"abideen/gemma-2b-openhermes"]
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained("google/gemma-7b")



import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from threading import Thread



class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = [29, 0]
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False
partial_message = ""
def predict(message, history):
    global partial_message
    prompt=f"""
###Below is an instruction that describes my question or task.
Write a response that appropriately completes the request:
{message}
### Response :

"""
    stop = StopOnTokens()
    history_transformer_format = history + [[message, ""]]
    model_inputs = tokenizer(prompt , return_tensors="pt")
    streamer = TextIteratorStreamer(tokenizer, timeout=200, skip_prompt=False , skip_special_tokens=False )
    generate_kwargs = dict(
        model_inputs,
        streamer=streamer,
        max_new_tokens=1024,
        top_p=0.95,
        top_k=50,
        temperature=0.85,
        num_beams=1,
        stopping_criteria=StoppingCriteriaList([stop]))
    t = Thread(target=model.generate,kwargs=generate_kwargs)
    t.start()

    partial_message = ""
    for new_token in streamer:
            partial_message += new_token
            yield partial_message

chatbot = gr.Chatbot(likeable=True,show_copy_button=True)
textbox = gr.Textbox (show_copy_button=True)
gr.ChatInterface(predict,chatbot=chatbot,textbox=textbox,stop_btn="Stop" ,retry_btn="Retry",concurrency_limit=1).launch(debug=True,share=True ,inline=False )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
